In [ ]:
#There's an Error when submmitting to the competition  ----------- Notebook Threw Exception
import sys 
sys.path.append("../input/notebookd97ba6b497/")

In [ ]:
# coding=utf-8
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

In [ ]:
epochs = 2  # 训练次数
batch_size = 32  # 批处理大小
num_workers = 0  # 多线程的数目
use_gpu = torch.cuda.is_available()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

root = '../input/petfinder-pawpularity-score/'
model_root = '../input/notebookd97ba6b497/pic_net_model.pt'
# set the path
train_pic_path = root + 'train'
test_pic_path = root + 'test'
train_csv_path = root + 'train.csv'
test_csv_path = root + 'test.csv'

In [ ]:
class FineTuneResnet50(nn.Module):
    def __init__(self, num_class=10):
        super(FineTuneResnet50, self).__init__()
        self.num_class = num_class
        resnet50_net = models.resnet50(pretrained=False)
        self.features = nn.Sequential(*list(resnet50_net.children())[:-1])
        self.classifier = nn.Linear(2048, self.num_class)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()

    def forward(self, x, y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y))
        return loss

In [ ]:
class PicSet(Dataset):
    def __init__(self, root, train=False):
        # 所有图片的绝对路径
        self.train=train
        self.imgs = os.listdir(root)
        self.root = root
        self.transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.csv_data = pd.read_csv(root + '.csv')


    def __getitem__(self, index):
        img_path = self.root + '/'+self.csv_data['Id'][index] + '.jpg'
        pil_img = Image.open(img_path)
        if self.transforms:
            data = self.transforms(pil_img)
        else:
            pil_img = np.asarray(pil_img)
            data = torch.from_numpy(pil_img)
        if self.train:
            label = torch.tensor(self.csv_data['Pawpularity'][index]).to(torch.float32)
            return {'pic': data, 'label': label}
        else:
            return {'pic': data}


    def __len__(self):
        return len(self.imgs)

In [ ]:
def load_pic(train_path, test_path):
    train_dataset = PicSet(train_path,train=True)
    pic_num = len(train_dataset)
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset,
                                                               [int(pic_num * 0.9), pic_num - int(pic_num * 0.9)])
    # print('train:', len(train_dataset), 'val:', len(val_dataset))

    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True,
                                               num_workers=num_workers)

    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             shuffle=False,
                                             num_workers=num_workers)

    test_dataset = PicSet(test_path)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                                              num_workers=num_workers)
    return train_loader, val_loader, test_loader

In [ ]:
if __name__ == '__main__':
    train_loader, val_loader, test_loader = load_pic(train_pic_path, test_pic_path)

    test_df = pd.read_csv(test_csv_path)
    test_score = []

    pic_net = FineTuneResnet50(num_class=1).to(device)
    
    pic_net.load_state_dict(torch.load(model_root))


    criterion = RMSELoss()
    optimizer_PIC = optim.Adam(pic_net.parameters(), lr=0.0001)

    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        each_train_losses = []
        each_val_losses = []

        pic_net.train()

        with tqdm(total=len(train_loader)) as t:
            for data in train_loader:

                pic_score = pic_net(data['pic'].to(device))

                total_score = pic_score

                loss = criterion(total_score, data['label'].to(device).view(-1, 1))

                each_train_losses.append(loss.item())

                optimizer_PIC.zero_grad()

                loss.backward()

                optimizer_PIC.step()

                t.set_description(f'Epoch [{epoch+1}/{epochs}]')
                t.set_postfix(loss=loss.item())
                t.update(1)

        pic_net.eval()

        for data in val_loader:
            
            pic_score = pic_net(data['pic'].to(device))
                
            # total_score = (pic_score + data_score) / 2
            total_score = pic_score
            
            loss = criterion(total_score, data['label'].to(device).view(-1, 1))

            each_val_losses.append(loss.item())

        val_losses.append(np.mean(each_val_losses))
        train_losses.append(np.mean(each_train_losses))

        # print('--------------Epoch %d --------------' % (epoch+1))
        print('Epoch: ', epoch+1, ', training loss:', np.mean(each_train_losses), ', validate loss: ',
              np.mean(each_val_losses))

    torch.save(pic_net.state_dict(), 'pic_net_model.pt')
    
    test_score = []

    for data in test_loader:
        
        pic_score = pic_net(data['pic'].to(device))
        
        pic_score = np.array(pic_score.cpu().detach().numpy())

        test_score.extend(list(map(float, np.mean(pic_score, axis=1))))
    

    submission = pd.DataFrame({"Id": test_df.Id.values, "Pawpularity": test_score})
    submission.to_csv("submission.csv", index=False)
#     print(submission)

In [ ]:
submission.head()